In [1]:
%config InlineBackend.figure_format='retina'

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns;sns.set_style('whitegrid',{'grid.color':'0.95'})

from linearmodels.iv import IV2SLS
from linearmodels.iv import IVLIML
from statsmodels.multivariate.cancorr import CanCorr
from weak_instr import weak_instr

In [2]:
poe5csv='http://www.principlesofeconometrics.com/poe5/data/csv/'
truf=pd.read_csv(poe5csv+'truffles.csv')
print(truf.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   p       30 non-null     float64
 1   q       30 non-null     float64
 2   ps      30 non-null     float64
 3   di      30 non-null     float64
 4   pf      30 non-null     float64
dtypes: float64(5)
memory usage: 1.3 KB
None


In [3]:
print(truf.describe(percentiles=[]).transpose())

    count       mean        std     min     50%      max
p    30.0  62.724000  18.723462  29.640  63.075  105.450
q    30.0  18.458333   4.613088   6.370  19.270   26.270
ps   30.0  22.022000   4.077237  15.210  22.685   28.980
di   30.0   3.526967   1.040803   1.525   3.708    5.125
pf   30.0  22.753333   5.329654  10.520  24.145   34.010


In [4]:
resq=smf.ols('q~ps+di+pf+1',data=truf).fit()
print(resq.summary(slim=True))

                            OLS Regression Results                            
Dep. Variable:                      q   R-squared:                       0.697
Model:                            OLS   Adj. R-squared:                  0.662
No. Observations:                  30   F-statistic:                     19.97
Covariance Type:            nonrobust   Prob (F-statistic):           6.33e-07
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.8951      3.243      2.434      0.022       1.228      14.562
ps             0.6564      0.143      4.605      0.000       0.363       0.949
di             2.1672      0.700      3.094      0.005       0.727       3.607
pf            -0.5070      0.121     -4.181      0.000      -0.756      -0.258

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.


In [5]:
resp=smf.ols('p~ps+di+pf+1',data=truf).fit()
print(resp.summary(slim=True))
truf['phat']=resp.predict()

                            OLS Regression Results                            
Dep. Variable:                      p   R-squared:                       0.889
Model:                            OLS   Adj. R-squared:                  0.876
No. Observations:                  30   F-statistic:                     69.19
Covariance Type:            nonrobust   Prob (F-statistic):           1.60e-12
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -32.5124      7.984     -4.072      0.000     -48.924     -16.101
ps             1.7081      0.351      4.868      0.000       0.987       2.429
di             7.6025      1.724      4.409      0.000       4.058      11.147
pf             1.3539      0.299      4.536      0.000       0.740       1.967

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.


In [6]:
res=smf.ols('q~phat+ps+di+1',data=truf).fit()
print(res.summary(slim=True))

                            OLS Regression Results                            
Dep. Variable:                      q   R-squared:                       0.697
Model:                            OLS   Adj. R-squared:                  0.662
No. Observations:                  30   F-statistic:                     19.97
Covariance Type:            nonrobust   Prob (F-statistic):           6.33e-07
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.2795      3.014     -1.420      0.168     -10.474       1.916
phat          -0.3745      0.090     -4.181      0.000      -0.559      -0.190
ps             1.2960      0.193      6.712      0.000       0.899       1.693
di             5.0140      1.241      4.039      0.000       2.462       7.566

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.


In [7]:
truf['Intercept']=1
res2sls=IV2SLS(dependent=truf.q,exog=truf[['Intercept','di','ps']],endog=truf.p
               ,instruments=truf.pf).fit(cov_type='unadjusted')
print(res2sls.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                      q   R-squared:                     -0.0239
Estimator:                    IV-2SLS   Adj. R-squared:                -0.1421
No. Observations:                  30   F-statistic:                    20.432
Date:                Wed, Jun 18 2025   P-value (F-stat)                0.0001
Time:                        09:25:29   Distribution:                  chi2(3)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -4.2795     5.1611    -0.8292     0.4070     -14.395      5.8361
di             5.0140     2.1259     2.3585     0.01

In [8]:
res2sls=IV2SLS(dependent=truf.q,exog=truf[['Intercept','di','ps']],endog=truf.p
               ,instruments=truf.pf).fit(cov_type='unadjusted',debiased='True')
print(res2sls.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                      q   R-squared:                     -0.0239
Estimator:                    IV-2SLS   Adj. R-squared:                -0.1421
No. Observations:                  30   F-statistic:                    5.9026
Date:                Wed, Jun 18 2025   P-value (F-stat)                0.0033
Time:                        09:25:29   Distribution:                  F(3,26)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -4.2795     5.5439    -0.7719     0.4471     -15.675      7.1161
di             5.0140     2.2836     2.1957     0.03

In [9]:
print(res2sls.first_stage)

    First Stage Estimation Results    
                                     p
--------------------------------------
R-squared                       0.8887
Partial R-squared               0.4417
Shea's R-squared                0.4417
Partial F-statistic             20.572
P-value (Partial F-stat)        0.0001
Partial F-stat Distn           F(1,26)
========================== ===========
Intercept                      -32.512
                             (-4.0721)
di                              7.6025
                              (4.4089)
ps                              1.7081
                              (4.8682)
pf                              1.3539
                              (4.5356)
--------------------------------------

T-stats reported in parentheses
T-stats use same covariance type as original model


In [10]:
weak_instr(1,1)


 Stock & Yogo (2005) Weak Instrument Critical Values
--------------------------------------------------------------
H0: Instruments are weak
                                 # of endogenous regressors: 1
                                 # of excluded instruments:  1
--------------------------------------------------------------
                                        5%    10%   20%    30%
2SLS relative bias:                    nan    nan   nan    nan
Fuller-k relative bias:              24.09  19.35 15.42  12.71
Fuller-k maximum bias:               23.81  19.40 15.39  12.76
--------------------------------------------------------------
                                       10%    15%   20%    25%
2SLS Size of nominal 5% Wald test:   16.38   8.96  6.66   5.53
LIML Size of nominal 5% Wald test:   16.38   8.96  6.66   5.53
--------------------------------------------------------------


In [11]:
res2sls=IV2SLS(dependent=truf.q,exog=truf[['Intercept','pf']]
               ,endog=truf.p,instruments=truf[['ps','di']]
              ).fit(cov_type='unadjusted',debiased='True')
print(res2sls.summary.tables[1])

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      20.033     1.2231     16.379     0.0000      17.523      22.542
pf            -1.0009     0.0825    -12.128     0.0000     -1.1702     -0.8316
p              0.3380     0.0249     13.563     0.0000      0.2869      0.3891


In [12]:
print(res2sls.first_stage)

    First Stage Estimation Results    
                                     p
--------------------------------------
R-squared                       0.8887
Partial R-squared               0.7614
Shea's R-squared                0.7614
Partial F-statistic             41.487
P-value (Partial F-stat)     8.117e-09
Partial F-stat Distn           F(2,26)
========================== ===========
Intercept                      -32.512
                             (-4.0721)
pf                              1.3539
                              (4.5356)
ps                              1.7081
                              (4.8682)
di                              7.6025
                              (4.4089)
--------------------------------------

T-stats reported in parentheses
T-stats use same covariance type as original model


In [13]:
fsh=pd.read_csv(poe5csv+'fultonfish.csv')
print(fsh[['lquan','lprice','mon','tue','wed','thu','stormy']].head())

      lquan    lprice  mon  tue  wed  thu  stormy
0  8.994421 -0.430783    1    0    0    0       1
1  7.707063  0.000000    0    1    0    0       1
2  8.350194  0.072321    0    0    1    0       0
3  8.656955  0.247139    0    0    0    1       1
4  7.844241  0.664327    0    0    0    0       1


In [14]:
print(fsh[['lquan','lprice','mon','tue','wed','thu','stormy']].
      describe(percentiles=[]).transpose())

        count      mean       std       min       50%       max
lquan   111.0  8.523430  0.741672  6.194406  8.621193  9.981374
lprice  111.0 -0.193681  0.381935 -1.107745 -0.206514  0.664327
mon     111.0  0.189189  0.393435  0.000000  0.000000  1.000000
tue     111.0  0.207207  0.407143  0.000000  0.000000  1.000000
wed     111.0  0.189189  0.393435  0.000000  0.000000  1.000000
thu     111.0  0.207207  0.407143  0.000000  0.000000  1.000000
stormy  111.0  0.288288  0.455020  0.000000  0.000000  1.000000


In [15]:
resq=smf.ols('lquan~mon+tue+wed+thu+stormy+1',data=fsh).fit()
print(resq.summary(slim=True))

                            OLS Regression Results                            
Dep. Variable:                  lquan   R-squared:                       0.193
Model:                            OLS   Adj. R-squared:                  0.155
No. Observations:                 111   F-statistic:                     5.034
Covariance Type:            nonrobust   Prob (F-statistic):           0.000356
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      8.8101      0.147     59.922      0.000       8.519       9.102
mon            0.1010      0.207      0.489      0.626      -0.308       0.510
tue           -0.4847      0.201     -2.410      0.018      -0.884      -0.086
wed           -0.5531      0.206     -2.688      0.008      -0.961      -0.145
thu            0.0537      0.201      0.267      0.790      -0.345       0.452
stormy        -0.3878      0.144     -2.698      0.0

In [16]:
resp=smf.ols('lprice~mon+tue+wed+thu+stormy+1',data=fsh).fit()
print(resp.summary(slim=True))

                            OLS Regression Results                            
Dep. Variable:                 lprice   R-squared:                       0.179
Model:                            OLS   Adj. R-squared:                  0.140
No. Observations:                 111   F-statistic:                     4.575
Covariance Type:            nonrobust   Prob (F-statistic):           0.000816
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2717      0.076     -3.557      0.001      -0.423      -0.120
mon           -0.1129      0.107     -1.052      0.295      -0.326       0.100
tue           -0.0411      0.105     -0.394      0.695      -0.248       0.166
wed           -0.0118      0.107     -0.111      0.912      -0.224       0.200
thu            0.0496      0.104      0.475      0.636      -0.157       0.257
stormy         0.3464      0.075      4.639      0.0

In [17]:
hypo='(mon=0,tue=0,wed=0,thu=0)'
print(f'F(4,105) = {resp.f_test(hypo).fvalue:.2}')
print(f'p-value  = {resp.f_test(hypo).pvalue.item(0):.4}')

F(4,105) = 0.62
p-value  = 0.6501


In [18]:
fsh['Intercept']=1
res2sls=IV2SLS(dependent=fsh.lquan
               ,exog=fsh[['Intercept','mon','tue','wed','thu']]
               ,endog=fsh.lprice
               ,instruments=fsh.stormy).fit(cov_type='unadjusted'
                                           ,debiased='True')
print(res2sls.first_stage)

    First Stage Estimation Results    
                                lprice
--------------------------------------
R-squared                       0.1789
Partial R-squared               0.1701
Shea's R-squared                0.1701
Partial F-statistic             21.517
P-value (Partial F-stat)     1.015e-05
Partial F-stat Distn          F(1,105)
========================== ===========
Intercept                      -0.2717
                             (-3.5569)
mon                            -0.1129
                             (-1.0525)
tue                            -0.0411
                             (-0.3937)
wed                            -0.0118
                             (-0.1106)
thu                             0.0496
                              (0.4753)
stormy                          0.3464
                              (4.6387)
--------------------------------------

T-stats reported in parentheses
T-stats use same covariance type as original model


In [19]:
weak_instr(1,1)


 Stock & Yogo (2005) Weak Instrument Critical Values
--------------------------------------------------------------
H0: Instruments are weak
                                 # of endogenous regressors: 1
                                 # of excluded instruments:  1
--------------------------------------------------------------
                                        5%    10%   20%    30%
2SLS relative bias:                    nan    nan   nan    nan
Fuller-k relative bias:              24.09  19.35 15.42  12.71
Fuller-k maximum bias:               23.81  19.40 15.39  12.76
--------------------------------------------------------------
                                       10%    15%   20%    25%
2SLS Size of nominal 5% Wald test:   16.38   8.96  6.66   5.53
LIML Size of nominal 5% Wald test:   16.38   8.96  6.66   5.53
--------------------------------------------------------------


In [20]:
print(res2sls.summary.tables[1])

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      8.5059     0.1662     51.189     0.0000      8.1764      8.8354
mon           -0.0254     0.2148    -0.1183     0.9061     -0.4513      0.4005
tue           -0.5308     0.2080    -2.5518     0.0122     -0.9432     -0.1183
wed           -0.5664     0.2128    -2.6620     0.0090     -0.9882     -0.1445
thu            0.1093     0.2088     0.5233     0.6018     -0.3047      0.5233
lprice        -1.1194     0.4286    -2.6115     0.0103     -1.9693     -0.2695


In [21]:
mroz=pd.read_csv(poe5csv+'mroz.csv')
mroz['Intercept']=1
mroz=mroz[mroz.lfp>0]
mroz['nwifeinc']=(mroz['faminc']-mroz['wage']*mroz['hours'])/1000
resivliml=IVLIML(dependent=mroz['hours'],exog=mroz[['Intercept','kidsl6','nwifeinc']]
                 ,endog=mroz[['mtr','educ']]
                 ,instruments=mroz[['mothereduc','fathereduc','exper']
                                  ]).fit(cov_type='unadjusted')
print(resivliml.summary.tables[1])

                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept   1.859e+04     3662.0     5.0759     0.0000   1.141e+04   2.577e+04
kidsl6         207.55     162.30     1.2789     0.2009     -110.54      525.65
nwifeinc      -104.94     20.565    -5.1028     0.0000     -145.25     -64.634
mtr         -1.92e+04     3980.2    -4.8230     0.0000    -2.7e+04   -1.14e+04
educ          -197.26     64.243    -3.0705     0.0021     -323.17     -71.346


In [22]:
print(resivliml.first_stage)

          First Stage Estimation Results         
                                   mtr       educ
-------------------------------------------------
R-squared                       0.6892     0.2792
Partial R-squared               0.1182     0.1994
Shea's R-squared                0.0618     0.1042
Partial F-statistic             19.129     35.525
P-value (Partial F-stat)     1.197e-11   1.11e-16
Partial F-stat Distn          F(3,422)   F(3,422)
========================== =========== ==========
Intercept                       0.8296     8.1762
                              (94.000)   (20.481)
kidsl6                          0.0156     0.7292
                              (2.8855)   (2.9845)
nwifeinc                       -0.0058     0.0530
                             (-29.167)   (5.8470)
mothereduc                     -0.0013     0.1560
                             (-1.7703)   (4.5732)
fathereduc                     -0.0020     0.1675
                             (-2.8259)   (5.1915)


In [23]:
varlist=mroz[['mtr','educ','mothereduc','fathereduc','exper']]
for v in varlist:
    res=sm.OLS(mroz[v],mroz[['Intercept','kidsl6','nwifeinc']]).fit()
    mroz[str(v+'r')]=res.resid
ccor=CanCorr(mroz[['mtrr','educr']],mroz[['mothereducr','fathereducr','experr']])
print(ccor.corr_test().summary())

                          Cancorr results
  Canonical Correlation Wilks' lambda Num DF  Den DF  F Value Pr > F
--------------------------------------------------------------------
0                0.4624        0.7409 6.0000 848.0000 22.8614 0.0000
1                0.2400        0.9424 2.0000 425.0000 12.9938 0.0000
--------------------------------------------------------------------
                                                                    
--------------------------------------------------------------------
Multivariate Statistics and F Approximations                        
----------------------------------------------------------------------
                         Value    Num DF    Den DF    F Value   Pr > F
----------------------------------------------------------------------
Wilks' lambda            0.7409   6.0000   846.0000   22.8075   0.0000
Pillai's trace           0.2714   6.0000   848.0000   22.1901   0.0000
Hotelling-Lawley trace   0.3330   6.0000   562.2257

In [24]:
n=resivliml.nobs
l=resivliml.first_stage.instr.shape
g=resivliml.first_stage.exog.shape
minccor=ccor.cancorr.min()
cd_stat=(n-(l[1]+g[1]))/l[1]*minccor**2/(1-minccor**2)
print(f'Cragg-Donald F-statistic: {cd_stat:.6}')

Cragg-Donald F-statistic: 8.60138


In [25]:
weak_instr(2,3)


 Stock & Yogo (2005) Weak Instrument Critical Values
--------------------------------------------------------------
H0: Instruments are weak
                                 # of endogenous regressors: 2
                                 # of excluded instruments:  3
--------------------------------------------------------------
                                        5%    10%   20%    30%
2SLS relative bias:                    nan    nan   nan    nan
Fuller-k relative bias:              10.83   8.96  7.18   6.15
Fuller-k maximum bias:               10.00   8.39  6.79   5.88
--------------------------------------------------------------
                                       10%    15%   20%    25%
2SLS Size of nominal 5% Wald test:   13.43   8.18  6.40   5.45
LIML Size of nominal 5% Wald test:    5.44   3.81  3.32   3.09
--------------------------------------------------------------


In [26]:
resivliml=IVLIML(dependent=mroz['hours'],exog=mroz[['Intercept','kidsl6','nwifeinc']]
                 ,endog=mroz[['mtr','educ']]
                 ,instruments=mroz[['mothereduc','fathereduc','exper']]
                 ,fuller=1
                ).fit(cov_type='unadjusted')
print(resivliml.summary)

                    IV-LIML(fuller(alpha=1)) Estimation Summary                     
Dep. Variable:                        hours   R-squared:                     -0.1746
Estimator:         IV-LIML(fuller(alpha=1))   Adj. R-squared:                -0.1857
No. Observations:                       428   F-statistic:                    37.319
Date:                      Wed, Jun 18 2025   P-value (F-stat)                0.0000
Time:                              09:25:30   Distribution:                  chi2(4)
Cov. Estimator:                  unadjusted                                         
                                                                                    
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept   1.816e+04     3539.3     5.1301     0.0000   1.122e+04   2.509e+04
kids